In [ ]:
import numpy as np
import OwnFrame as fr

In [ ]:
f = open('shakespear.txt', 'r')
raw = f.read()
f.close()

In [ ]:
vocab = list(set(raw)) # литерали список символов

In [ ]:
word2index = {}
for i, word in enumerate(vocab):
    word2index[word] = i

indices = np.array(list(map(lambda x: word2index[x], raw)))


In [ ]:
embed = fr.Embedding(vocab_size=len(vocab), dim=512)
model = fr.RNNCell(n_input=512, n_hidden=512, n_output=len(vocab))

criterion = fr.CrossEntropyLoss()
optim = fr.SGD(parameters=model.get_parameters() + embed.get_parameters(), alpha=0.05)


Усечнное обратное распространение

In [ ]:
batch_size = 32
bptt = 16
n_batches = int(indices.shape[0] / batch_size)

In [ ]:
# усечение набора данных до размера, кратного поизведению batch_size и n_batches
trimmed_indices = indices[:n_batches * batch_size]

# изменение формы набора данных так, чтобы каждый столбец представлял сегмент начального массива индексов
batched_indices = trimmed_indices.reshape(batch_size, n_batches)
batched_indices = batched_indices.transpose()

In [ ]:
print(raw[0:5])
print(indices[0:5])

In [ ]:
print(batched_indices[0:5])

In [ ]:
input_batched_indices = batched_indices[0:-1]
target_batched_indices = batched_indices[1:] # выходные индексы - входные индексы со смещением на одну строку

In [ ]:
print(input_batched_indices)

In [ ]:
print(target_batched_indices)

In [ ]:
n_bptt = int((n_batches - 1) / bptt)
input_batches = input_batched_indices[:n_bptt * bptt]
input_batches = input_batches.reshape(n_bptt, bptt, batch_size)
target_batches = target_batched_indices[:n_bptt * bptt]
target_batches = target_batches.reshape(n_bptt, bptt, batch_size)

In [ ]:
print(input_batches[0][0:5])

In [ ]:
print(target_batches[0][0:5])

In [ ]:
def generate_sample(n=30, init_char=' '):
    s = ""
    hidden = model.init_hidden(batch_size=1)
    input = fr.Tensor(np.array([word2index[init_char]]))
    for i in range(n):
        rnn_input = embed.forward(input)
        output, hidden = model.forward(input=rnn_input, hidden=hidden)
        
        output.data *= 10 # температура для отбора образцов, чем больше, тем выше вероятность отбора
        temp_dist = output.softmax()
        temp_dist /= temp_dist.sum()
        
        # образцы из прогноза
        m = (temp_dist > np.random.rand()).argmax()
        c = vocab[m]
        input = fr.Tensor(np.array([m]))
        s += c
    return s

In [ ]:
import sys


def train(iterations=100):
    for iter in range(iterations):
        total_loss = 0
        n_loss = 0
        
        hidden = model.init_hidden(batch_size=batch_size)
        for batch_i in range(len(input_batches)):
            hidden = fr.Tensor(hidden.data, autograd=True)
            loss = None
            losses = list()
            for t in range(bptt):
                input = fr.Tensor(input_batches[batch_i][t], autograd=True)
                rnn_input = embed.forward(input=input)
                output, hidden = model.forward(input=rnn_input, hidden=hidden)
                
                target = fr.Tensor(target_batches[batch_i][t], autograd=True)
                batch_loss = criterion.forward(output, target)
                losses.append(batch_loss)
                
                if t == 0:
                    loss = batch_loss
                else:
                    loss = loss + batch_loss
            
            for loss in losses:
                ""
            loss.backward()
            optim.step()
            total_loss += loss.data

            log = "\r Iter:" + str(iter)
            log += " - Batch "+str(batch_i+1)+"/"+str(len(input_batches))
            log += " - Loss:" + str(np.exp(total_loss / (batch_i+1)))
            
            if batch_i == 0:
                log += " - " + generate_sample(n=70, init_char='\n').replace("\n"," ")
                
            if batch_i % 10 == 0 or batch_i-1 == len(input_batches):
                sys.stdout.write(log)
        optim.alpha *= 0.99
        print()
            ##### дописать вывод

In [ ]:
train()